In [2]:
%load_ext autoreload
%autoreload 2
import torch
from transformers import BertTokenizer
import csv
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel
import time
import datetime
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup
import sys
import argparse
import os
from collections import defaultdict
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from typing import List, Dict
from torch.utils.data import Dataset
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from typing import List
import tqdm
import csv
import json
import faiss  

In [3]:
df = pd.read_csv("results.covid_dataset.all.tsv", sep = "\t")

In [4]:
ids, sents = df["sentence_id"].tolist(), df["sentence_text"].tolist()

In [5]:
len(sents)

2318939

In [6]:
sents[0]

'Sequences in bold indicate codons requiring multiple nucleotide substitutions .'

In [7]:
sents[5]

'These analytic weights were then multiplied by the appropriate adjustment factors as described above .'

In [8]:
class BertEncoder(object):
    
    def __init__(self, device = 'cpu'):
        
        self.tokenizer = BertTokenizer.from_pretrained('/media/shauli/Elements/current_projects/AI2/Clustering/scibert_scivocab_uncased/vocab.txt')
        self.model = BertModel.from_pretrained('/media/shauli/Elements/current_projects/AI2/Clustering/scibert_scivocab_uncased/')
            
        self.model.eval()
        self.model.to(device)
        self.device = device
        self.pad_token = self.tokenizer.convert_tokens_to_ids([self.tokenizer.pad_token])[0]
        
    def tokenize_and_pad(self, texts: List[str]):
        
        indexed_texts = [self.tokenizer.encode(text, add_special_tokens=True, max_length = 512) for text in texts] #
        max_len = max(len(text) for text in indexed_texts)
        indexed_texts = [text + [self.pad_token] * (max_len - len(text)) for text in indexed_texts]
        idx_tensor = torch.LongTensor(indexed_texts).to(self.device)
        att_tensor = idx_tensor != self.pad_token
        
        return idx_tensor, att_tensor
    
    def encode(self, sentences: List[str], sentence_ids: List[str], batch_size: int, strategy: str = "cls", fname=""):
        assert len(sentences) == len(sentence_ids)
        
        with open(fname, "w", encoding = "utf-8") as f:
            
            for batch_idx in tqdm.tqdm_notebook(range(0, len(sentences), batch_size), total = len(sentences)//batch_size):
            
                batch_sents = sentences[batch_idx: batch_idx + batch_size]
                batch_ids = sentence_ids[batch_idx: batch_idx + batch_size]
                assert len(batch_sents) == len(batch_ids)
                
                idx, att_mask = self.tokenize_and_pad(batch_sents)
            
                with torch.no_grad():
                    outputs = self.model(idx, attention_mask = att_mask)
                    last_hidden = outputs[0]
                
                    if strategy == "cls":
                        h = last_hidden[:, 0, ...]
                    elif strategy == "mean-cls":
                        h = torch.cat([last_hidden[:, 0, ...], torch.mean(last_hidden, axis = 1)], axis = 1)
                    elif strategy == "mean-cls-max":
                       h_max = torch.max(last_hidden, axis = 1).values
                       h = torch.cat([last_hidden[:, 0, ...], torch.mean(last_hidden, axis = 1), h_max], axis = 1)
                    elif strategy == "mean":
                        h = torch.mean(last_hidden, axis = 1)
                    elif strategy == "median":
                        h = torch.median(last_hidden, axis = 1).values
                    elif strategy == "max":
                        h = torch.max(last_hidden, axis = 1).values
                    elif strategy == "min":
                        h = torch.min(last_hidden, axis = 1).values
            
                batch_np = h.detach().cpu().numpy()
                assert len(batch_np) == len(batch_sents)
                
                sents_states_ids = zip(batch_sents, batch_np, batch_ids)
                for sent, vec, sent_id in sents_states_ids:
                    
                    vec_str = " ".join(["%.4f" % x for x in vec])
                    sent_dict = {"text": sent, "vec": vec_str, "id": sent_id}
                    f.write(json.dumps(sent_dict) + "\n")

In [9]:
encoder = BertEncoder("cuda:0")

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [10]:
h = encoder.encode(sents, ids, batch_size = 32, strategy = "mean-cls-max", fname="output.mean-cls-max.jsonl")

KeyboardInterrupt: 

In [10]:
n = 50000
data = []

with open("output.mean-cls.jsonl", "r", encoding = "utf-8") as f:
    i = 0
    for line in f:
            data_dict = eval(line)
            data_dict["vec"] = np.array([float(x) for x in data_dict["vec"].split(" ")]).astype("float32")
            data.append(data_dict)
            i += 1
            if i > n: break

In [36]:
vecs_sample = np.array([d["vec"] for d in data])
sents_sample =[d["text"] for d in data]
pca = PCA(n_components = 0.98)
vecs_sample_pca = pca.fit_transform(vecs_sample).astype("float32")

index = faiss.IndexFlatIP(vecs_sample_pca.shape[1])
index.add(np.ascontiguousarray(vecs_sample_pca/np.linalg.norm(vecs_sample_pca, axis = 1, keepdims = True)))
faiss.write_index(index, "index.faiss")

In [43]:
vecs = np.array([index.reconstruct(i) for i in range(100)])

In [44]:
k = 15
spread_idx = np.array([i for i,sent in enumerate(sents_sample) if "economic" in sent])
spread_mean = np.mean(vecs_sample[spread_idx], axis = 0)
spread_mean_pca = pca.transform([spread_mean])
D, I = index.search(np.ascontiguousarray(spread_mean_pca), 10)

In [39]:
spread_idx.shape

(140,)

In [40]:
sents[spread_idx[3]]

'[ 62 , 63 ] The economic condition may contribute to a false diagnosis of microcephaly , and postdiagnosis follow-up is necessary to make a definite diagnosis or complete correct diagnosis , considering the limitation of the operational definition in the size of head circumference .'

In [35]:
for i in I.squeeze():
    print(sents_sample[i])
    print("===========================================================")

3 . the current capacity of local health care systems to detect and respond to importation and transmission of EVD , and their variation across the region ( health care system characteristics ) ; and 4 . plausible interventions to support local health care systems to detect and respond to importation and transmission of EVD ( interventions ) .
This has reignited concerns about shifting transmission dynamics of communicable diseases , including how climate change is likely to affect their global spread , and it has reiterated the need for political action to meet challenges in the near future .
Vaccination of these animals would cost at least US$ 373 000 per year ( US$ 1.1 per vaccine ) ; a significant financial burden to small-scale farmers who rely on livestock sales to pay for house maintenance and childhood education and for local governments that have not previously been affected by rabies .
With depleted numbers of experienced health care workers in EVD-affected countries resultin

In [ ]:
sents[1]

In [20]:
index.add(np.random.rand(10,175).astype("float32"))

In [18]:
vecs_sample_pca.shape

(5001, 175)